# 우울증 판별을 위한 얼굴 표정 이진 분류 모델

이 노트북에서는 주어진 얼굴 표정 이미지를 이용해 **우울증 여부**를 판별하는 이진 분류 모델을 개발합니다. 프로젝트의 최종 목표는 `불안(anxiety)`, `상처(hurt)`, `슬픔(sadness)` 감정에 해당하는 이미지를 "우울 관련(1)"로, 그 외의 감정(`분노(anger)`, `기쁨(joy)`, `중립(neutral)`, `당황(surprise)`)을 "비우울 관련(0)"으로 분류하는 것입니다.

데이터는 다음과 같은 구조를 가집니다:
```
data/
  train/
    train_image/        # 훈련 이미지 폴더 (감정별 하위 폴더)
    train_label/        # 훈련 라벨(JSON 파일)
  vali/
    vali_image/         # 검증 이미지 폴더
    vali_label/         # 검증 라벨(JSON 파일)
```
이 노트북에서는 이러한 구조를 기반으로 PyTorch 데이터셋을 정의하고, 이미지 전처리/증강을 수행하며, 사전 학습된 모델(ResNet50)을 미세조정(fine-tuning)하여 우울증 여부를 분류하는 과정을 단계별로 구현합니다. 중간 중간에 각 단계의 개념을 **백지상태에서 설명하듯이 전부 분해해서 설명**하며, 어려운 용어는 따로 정의합니다. 또한 클래스 불균형을 해결하기 위해 **클래스 가중치(class weight)**를 적용하는 방법을 다룹니다.


## 1. 패키지 불러오기와 기본 설정

먼저 모델 학습에 필요한 라이브러리들을 불러옵니다. PyTorch(`torch`), 이미지 전처리를 위한 `torchvision`, 데이터 분석을 위한 `pandas`와 `numpy`, 시각화를 위한 `matplotlib` 및 기타 유틸리티 패키지를 사용합니다. `device` 변수는 학습을 GPU에서 수행할 수 있도록 GPU(CUDA) 사용 가능 여부를 확인합니다.

- **PyTorch(Torch)**: 파이썬 기반의 딥러닝 라이브러리로, 텐서 연산 및 자동 미분 기능을 제공합니다.
- **torchvision**: 이미지 데이터셋과 이미지 전처리 함수들을 제공하는 PyTorch의 서브패키지입니다.
- **numpy**: 다차원 배열 연산을 위한 기본 패키지입니다.
- **pandas**: 테이블 형태의 데이터를 다루는 데 유용한 패키지입니다.
- **matplotlib**: 그래프 그리기를 위한 시각화 라이브러리입니다.


In [ ]:
import os
import json
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torchvision.transforms as transforms
import torchvision.models as models

# 현재 사용 가능한 장치를 확인합니다. GPU가 있으면 GPU를 사용하고, 없으면 CPU를 사용합니다.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


## 2. 감정 레이블 정의

주어진 데이터는 7개의 감정 레이블을 갖습니다. 우리는 이 7개 감정을 **우울 관련(1)**과 **비우울 관련(0)** 두 개의 클래스로 재분류합니다. 이를 위해 감정별로 한국어 레이블과 함께 매핑 딕셔너리를 정의하고, 우울 클래스에 해당하는 감정 목록(`DEPRESSION_EMOTIONS`)과 비우울 클래스에 해당하는 감정 목록(`NON_DEPRESSION_EMOTIONS`)을 만듭니다.

- **분노(anger)**, **기쁨(joy)**, **중립(neutral)**, **당황(surprise)**: 비우울 관련(0)
- **불안(anxiety)**, **상처(hurt)**, **슬픔(sadness)**: 우울 관련(1)


In [ ]:
# 원본 감정 이름과 한국어 표기를 매핑합니다.
EMOTIONS = {
    'anger': '분노',
    'anxiety': '불안',
    'hurt': '상처',
    'joy': '기쁨',
    'neutral': '중립',
    'sadness': '슬픔',
    'surprise': '당황'
}

# 우울과 비우울 클래스를 지정합니다.
DEPRESSION_EMOTIONS = ['anxiety', 'hurt', 'sadness']  # 레이블 1
NON_DEPRESSION_EMOTIONS = ['anger', 'joy', 'neutral', 'surprise']  # 레이블 0


## 3. 데이터셋 클래스 작성

`torch.utils.data.Dataset`을 상속하여 커스텀 데이터셋 클래스를 만듭니다. 이 클래스는 이미지 파일 경로와 라벨 정보를 읽어와서 `__getitem__` 메서드에서 한 샘플씩 반환하는 역할을 합니다.

### 3.1 라벨 JSON 파싱

훈련/검증 데이터의 레이블은 각각 JSON 파일로 제공됩니다. 각 JSON 파일에는 해당 감정 폴더의 이미지 파일 이름과 함께 라벨이 포함되어 있다고 가정합니다. 파싱 함수는 폴더 내 모든 JSON을 읽어 이미지 경로와 감정(텍스트 레이블)을 모으고, 이를 우울/비우울 이진 라벨로 변환합니다.

### 3.2 이미지 불러오기와 전처리

PyTorch에서 이미지를 로드하려면 `PIL.Image` 모듈을 사용하거나 `torchvision.io.read_image` 등을 활용할 수 있습니다. 여기서는 `PIL.Image.open`을 사용한 후, 지정한 전처리(transform)를 적용합니다. 전처리 과정에는 다음이 포함됩니다:

1. **리사이즈(Resize)**: 모든 이미지를 224×224 픽셀로 통일합니다.
2. **데이터 증강(Data Augmentation)**: 학습 데이터에 한해 무작위 좌우 뒤집기, 약간의 회전, 색조 변환 등으로 모델의 일반화 성능을 높입니다.
3. **Tensor 변환과 정규화**: 이미지를 PyTorch 텐서로 변환하고, 픽셀 값을 0~1 사이로 스케일링합니다. 또한 사전 학습된 모델에 맞는 평균과 표준편차를 사용해 정규화합니다.


In [ ]:
from PIL import Image

def parse_label_files(label_dir, image_dir):
    """
    주어진 레이블 폴더에서 모든 JSON 파일을 읽어 이미지 경로와 우울/비우울 레이블을 반환합니다.

    Args:
        label_dir (str): JSON 파일들이 있는 폴더 경로
        image_dir (str): 이미지 폴더의 상위 경로

    Returns:
        list of tuples: (image_path, binary_label)
    """
    samples = []
    json_files = sorted(glob(os.path.join(label_dir, '*.json')))

    for json_file in json_files:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # JSON의 구조가 { 'file_name': emotion } 형태라고 가정
        for file_name, emotion in data.items():
            # 이미지 경로 구성 (emotion 이름과 폴더가 동일하다고 가정)
            img_path = os.path.join(image_dir, emotion, file_name)
            # 우울 관련 감정은 1, 비우울은 0
            label = 1 if emotion in DEPRESSION_EMOTIONS else 0
            samples.append((img_path, label))

    return samples

class EmotionDataset(Dataset):
    """
    얼굴 이미지와 우울/비우울 레이블을 포함하는 커스텀 데이터셋 클래스입니다.
    """
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)


## 4. 데이터 전처리(변환) 정의

`torchvision.transforms`를 사용하여 학습용과 검증용 전처리를 각각 정의합니다. 학습 데이터에는 데이터 증강을 포함하고, 검증 데이터에는 리사이즈와 텐서 변환만 적용합니다.

- **RandomHorizontalFlip**: 50% 확률로 이미지를 좌우로 뒤집어 모델이 좌우 대칭에 덜 민감하도록 합니다.
- **RandomRotation**: ±10도 범위에서 이미지를 회전시켜 다양한 각도의 얼굴을 학습합니다.
- **ColorJitter**: 밝기, 대비, 채도, 색조를 랜덤하게 조정하여 조명 변화에 견딜 수 있게 합니다.
- **Resize**: 입력 이미지를 224×224 픽셀로 변경합니다.
- **ToTensor**: `PIL.Image`를 `[0, 1]` 범위의 PyTorch 텐서로 변환합니다.
- **Normalize**: 이미지의 픽셀 값을 평균과 표준편차로 정규화하여 학습을 안정화합니다. 여기서는 ImageNet 데이터셋에서 학습된 사전 학습 모델과 동일한 통계를 사용합니다.


In [ ]:
# ImageNet 통계 (사전 학습 모델과 호환)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

vali_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])


## 5. 데이터셋 인스턴스 생성과 클래스 불균형 처리

### 5.1 데이터셋 로딩
앞서 정의한 `parse_label_files` 함수를 사용하여 훈련 데이터와 검증 데이터의 (이미지 경로, 라벨) 리스트를 생성합니다. 실제 경로는 프로젝트 구조에 맞게 수정해야 합니다. 예를 들어, `train_label_dir`는 `data/train/train_label` 폴더의 경로이고, `train_image_dir`는 `data/train/train_image` 폴더의 경로입니다.

### 5.2 클래스 불균형(imbalance) 문제
감정 데이터의 분포가 균일하지 않기 때문에, 우울/비우울 레이블의 비율도 불균형할 수 있습니다. 이 문제를 해결하기 위해 각 클래스의 **가중치(weights)**를 계산하여 손실 함수에 적용하거나, `WeightedRandomSampler`를 사용하여 각 배치에서 클래스 비율을 균형 있게 만듭니다.

아래에서는 각 클래스의 샘플 수를 세어 클래스 가중치를 계산하고, 해당 가중치를 `WeightedRandomSampler`에 사용합니다.


In [ ]:
# 데이터 경로 설정 (실제 프로젝트 구조에 맞게 수정하세요)
base_dir = 'data'  # 예: C:/aug-08month_project5/hwa_in/data
train_label_dir = os.path.join(base_dir, 'train', 'train_label')
train_image_dir = os.path.join(base_dir, 'train', 'train_image')
vali_label_dir = os.path.join(base_dir, 'vali', 'vali_label')
vali_image_dir = os.path.join(base_dir, 'vali', 'vali_image')

# 레이블 파일에서 (경로, 라벨) 목록 추출
train_samples = parse_label_files(train_label_dir, train_image_dir)
vali_samples = parse_label_files(vali_label_dir, vali_image_dir)

# 데이터셋 인스턴스 생성
train_dataset = EmotionDataset(train_samples, transform=train_transform)
vali_dataset = EmotionDataset(vali_samples, transform=vali_transform)

# 클래스 불균형을 고려한 가중치 계산
labels = [label for _, label in train_samples]
class_sample_count = np.bincount(labels)
# 클래스가 0 또는 1만 존재한다고 가정
class_weights = 1. / class_sample_count
sample_weights = [class_weights[label] for label in labels]

# WeightedRandomSampler 생성: 배치마다 클래스가 균형 있게 추출되도록 도와줌
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# DataLoader 생성
batch_size = 32  # GPU 메모리에 따라 조절
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=4)
vali_loader = DataLoader(vali_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# 클래스 비율 출력 (확인용)
class_counts = { '비우울(0)': int(class_sample_count[0]), '우울(1)': int(class_sample_count[1]) }
class_counts


## 6. 모델 정의와 손실 함수 설정

딥러닝 모델로는 **ResNet50**을 사용합니다. ResNet(Residual Network)은 깊은 신경망에서 학습이 어려워지는 문제를 잔차 연결(residual connection)로 해결한 아키텍처입니다. 사전 학습(pretrained)된 모델을 불러온 뒤, 마지막 완전연결층(fc)을 우리의 이진 분류 과제에 맞게 수정합니다.

- **사전 학습(pretrained)**: 대규모 데이터셋(ImageNet)에서 이미 학습된 모델의 가중치를 초기값으로 사용함으로써, 적은 데이터에서도 더 빠른 수렴과 높은 성능을 기대할 수 있습니다.
- **`nn.CrossEntropyLoss`**: 소프트맥스(softmax)와 음의 로그우도 손실을 결합한 손실 함수로, 다중 클래스 분류와 2진 분류 모두에 사용됩니다. 클래스 불균형을 완화하기 위해 `weight` 인자로 클래스 가중치를 전달합니다.


In [ ]:
# 사전 학습된 ResNet50 모델 불러오기
model = models.resnet50(pretrained=True)

# 마지막 완전연결층의 입력 특징 수를 구함
num_ftrs = model.fc.in_features
# 2개 클래스(우울/비우울)에 맞게 출력 뉴런 수를 변경
model.fc = nn.Linear(num_ftrs, 2)

# 모델을 GPU/CPU에 할당
model = model.to(device)

# 클래스 가중치를 텐서로 변환하여 손실 함수에 전달
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# 옵티마이저 설정 (Adam 사용)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


## 7. 학습과 검증 루프 정의

모델 학습은 여러 에폭(epoch) 동안 훈련 데이터 전체를 반복하면서 수행됩니다. 각 에폭마다 훈련 손실과 정확도, 검증 손실과 정확도를 기록하여 모델의 성능 변화를 모니터링합니다. 에폭이 진행될수록 검증 손실이 감소하고 정확도가 증가하면 모델이 잘 학습되고 있다는 신호입니다.

### 주요 용어 설명
- **에폭(epoch)**: 훈련 데이터셋 전체를 한 번 통과하는 과정입니다.
- **배치(batch)**: 전체 데이터셋을 작게 나눈 묶음입니다. 한 번에 메모리에 올려 처리할 수 있는 데이터의 양입니다.
- **정방향 패스(forward pass)**: 입력 데이터를 모델에 통과시켜 예측값을 얻는 과정입니다.
- **역방향 패스(backpropagation)**: 예측값과 실제값의 차이를 계산한 손실을 기준으로 모델의 가중치를 업데이트하기 위해 미분을 수행하는 과정입니다.
- **옵티마이저(optimizer)**: 계산된 기울기(gradient)를 바탕으로 가중치를 업데이트하는 알고리즘입니다. 여기서는 Adam을 사용합니다.


In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()  # 모델을 학습 모드로 설정 (Dropout, BatchNorm 등이 활성화)
    running_loss = 0.0
    running_corrects = 0
    total = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # 기울기 초기화
        optimizer.zero_grad()

        # 정방향 패스
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 예측 결과
        _, preds = torch.max(outputs, 1)

        # 역방향 패스
        loss.backward()
        optimizer.step()

        # 통계 업데이트
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects.double() / total
    return epoch_loss, epoch_acc.item()

def evaluate(model, dataloader, criterion, device):
    model.eval()  # 모델을 평가 모드로 설정 (Dropout, BatchNorm 비활성)
    running_loss = 0.0
    running_corrects = 0
    total = 0

    # 평가 시에는 기울기 계산을 하지 않음
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = running_corrects.double() / total
    return epoch_loss, epoch_acc.item()


## 8. 모델 학습 실행

위에서 정의한 함수들을 이용해 모델을 여러 에폭 동안 학습합니다. 각 에폭마다 훈련 손실/정확도와 검증 손실/정확도를 출력합니다. 에폭 수(`num_epochs`)는 프로젝트 요구사항과 시간에 따라 조절할 수 있습니다. 초기에는 5~10 에폭 정도로 시작한 뒤, 모델의 성능을 보며 조정할 수 있습니다.


In [ ]:
num_epochs = 10  # 필요에 따라 조절

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    vali_loss, vali_acc = evaluate(model, vali_loader, criterion, device)

    print(f'Epoch {epoch}/{num_epochs}: ',
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc*100:.2f}%',
          f'| Val Loss: {vali_loss:.4f}, Val Acc: {vali_acc*100:.2f}%')


## 9. 예측 및 혼동 행렬 시각화

학습이 완료된 모델의 성능을 조금 더 자세히 분석하기 위해 검증 데이터에 대한 혼동 행렬(confusion matrix)을 시각화합니다. 혼동 행렬은 각 클래스(여기서는 우울/비우울)에 대해 모델이 얼마나 정확히 예측했는지를 보여줍니다.

- **True Positive (TP)**: 실제 우울 샘플을 우울로 정확히 예측한 경우
- **False Positive (FP)**: 실제 비우울 샘플을 우울로 잘못 예측한 경우
- **True Negative (TN)**: 실제 비우울 샘플을 비우울로 정확히 예측한 경우
- **False Negative (FN)**: 실제 우울 샘플을 비우울로 잘못 예측한 경우


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds, labels=[0, 1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['비우울', '우울'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix (Validation Set)')
    plt.show()

# 혼동 행렬 출력 (학습 완료 후 실행)
# plot_confusion_matrix(model, vali_loader, device)


## 10. Grad-CAM을 이용한 시각화

모델이 이미지의 어느 부분에 주목하여 우울/비우울을 판단하는지 시각적으로 확인하기 위해 **Grad-CAM**(Gradient-weighted Class Activation Mapping)을 적용합니다. Grad-CAM은 특정 클래스에 대한 출력을 기준으로 마지막 합성곱 층의 특성맵(feature map)과 그라디언트를 결합하여 중요 영역을 강조한 히트맵을 생성합니다.

아래 코드에서는 ResNet50의 마지막 합성곱 층인 `model.layer4`에 후크(hook)를 등록하여 forward 패스에서의 feature map과 backward 패스에서의 gradient를 저장한 뒤, 이를 이용해 히트맵을 생성하고 원본 이미지 위에 오버레이합니다. 검증 데이터의 첫 번째 샘플을 예시로 사용하지만, 원하는 다른 이미지에 적용할 수 있습니다.


In [ ]:
import matplotlib.cm as cm
import numpy as np
import matplotlib.pyplot as plt

def generate_gradcam(model, input_tensor, target_class):
    model.eval()
    feature_maps = []
    gradients = []

    def forward_hook(module, input, output):
        feature_maps.append(output.detach())
    def backward_hook(module, grad_in, grad_out):
        gradients.append(grad_out[0].detach())
    
    # 마지막 합성곱 층(layer4)에 후크 등록
    handle_f = model.layer4.register_forward_hook(forward_hook)
    handle_b = model.layer4.register_backward_hook(backward_hook)

    # forward pass
    output = model(input_tensor.unsqueeze(0))
    score = output[0, target_class]
    # backward pass
    model.zero_grad()
    score.backward()

    # 후크 해제
    handle_f.remove()
    handle_b.remove()

    # gradients와 feature_maps는 리스트로 저장되므로 첫 번째 요소 사용
    grads = gradients[0][0]  # shape: [C, H, W]
    fmap = feature_maps[0][0]  # shape: [C, H, W]

    # 각 채널별로 gradient를 평균내어 가중치 계산
    weights = grads.mean(dim=(1, 2))
    cam = torch.zeros(fmap.shape[1:], dtype=fmap.dtype).to(fmap.device)
    for i, w in enumerate(weights):
        cam += w * fmap[i]
    
    cam = torch.relu(cam)
    cam = cam - cam.min()
    cam = cam / (cam.max() + 1e-8)
    cam = cam.cpu().numpy()
    # 입력 이미지 크기로 리사이즈
    cam = np.array(Image.fromarray(cam).resize((input_tensor.size(2), input_tensor.size(1))))
    return cam


def show_gradcam_on_image(input_tensor, cam_mask, mean=IMAGENET_MEAN, std=IMAGENET_STD):
    # 입력 이미지는 정규화되어 있으므로 원래 스케일로 되돌림
    img = input_tensor.cpu().permute(1,2,0).numpy()
    img = img * np.array(std)[None, None, :] + np.array(mean)[None, None, :]
    img = np.clip(img, 0, 1)
    
    heatmap = cm.jet(cam_mask)[..., :3]  # RGBA 중 RGB만
    # 히트맵과 원본 이미지 합성
    overlay = heatmap * 0.4 + img
    overlay = overlay / overlay.max()
    
    plt.figure(figsize=(6,3))
    plt.subplot(1,2,1)
    plt.title('Original')
    plt.imshow(img)
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.title('Grad-CAM Overlay')
    plt.imshow(overlay)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# 예시: 검증 데이터 첫 번째 샘플에 Grad-CAM 적용
test_iter = iter(vali_loader)
example_inputs, example_labels = next(test_iter)
example_input = example_inputs[0].to(device)
example_label = example_labels[0].item()

# Grad-CAM 생성
cam_mask = generate_gradcam(model, example_input, example_label)
# 시각화
show_gradcam_on_image(example_input.cpu(), cam_mask)
